In [ ]:
import matplotlib.pyplot as plt
import torch
from typing import List
import pandas as pd

import nnts
import nnts.data
import nnts.experiments
import nnts.models
import nnts.torch.data.preprocessing as preprocessing
import nnts.torch.models
import nnts.torch.models.trainers as trainers
import nnts.metrics
import nnts.torch.data
import nnts.torch.data.datasets
import nnts.loggers
import covs 
import scipy
import os
import nnts.pandas

In [ ]:
data_path = "data"
model_name = "seg-lstm"
dataset_name = "weather"
results_path = "nb-results"
metadata_path = os.path.join(data_path, f"informer.json")
metadata = nnts.data.metadata.load(dataset_name, path=metadata_path)
datafile_path = os.path.join(data_path, metadata.filename)
PATH = os.path.join(results_path, model_name, metadata.dataset)

df_orig = pd.read_csv(datafile_path)
params = nnts.models.Hyperparams()
splitter = nnts.pandas.FixedSizeSplitter()

nnts.loggers.makedirs_if_not_exists(PATH)

In [ ]:
df_orig = df_orig.rename({"WetBulbCelsius": "y", "date": "ds"}, axis="columns")
df_orig["unique_id"] = "T1"

In [ ]:
# print correlation with target
cols = df_orig.columns.tolist()
cols.remove("ds")
cols.remove("unique_id")
cols.remove("y")
for i in range(len(cols)):
    pearson = scipy.stats.pearsonr(df_orig["y"], df_orig[cols[i]])
    print(cols[i], pearson[0])

In [ ]:
#plot correlation with various leads
pearson_list = []
for i in range(192):
    pearson = scipy.stats.pearsonr(
        df_orig["y"], df_orig.shift(i).bfill()["StationPressure"]
    )
    pearson_list.append(pearson[0])
plt.plot(pearson_list)

In [ ]:
df_orig["y_lead_1"] = df_orig["StationPressure"].shift(35)

In [ ]:
# split lengths as per informer
trn_length = int(24 * 365.25 * 2)
val_test_length = int(24 * 365.25 * (10 / 12))

In [ ]:
scenario_list: List[nnts.experiments.CovariateScenario] = []

# Add the baseline scenarios
for seed in [42, 43, 44, 45, 46]:
    scenario_list.append(
        nnts.experiments.CovariateScenario(metadata.prediction_length, error=0.0, covariates=0, seed=seed)
    )


scenario_list = [
    nnts.experiments.CovariateScenario(
        metadata.prediction_length, error=0.0, covariates=1, seed=42
    ),
    nnts.experiments.CovariateScenario(1, error=0.0, covariates=1, seed=42),
    nnts.experiments.CovariateScenario(1, error=0.0, covariates=0, seed=42),
]
scenario_list = [
    nnts.experiments.CovariateScenario(
        metadata.prediction_length, error=0.0, covariates=0, seed=42
    )
]

In [ ]:
def prepare(data, scenario):
    pearson = 0
    conts = []
    noise = 0
    if scenario.covariates > 0:
        pearson = covs.calculate_pearson(data)
        conts.append("y_lead_1")
    data = data.dropna()
    scenario.conts = conts
    scenario.pearson = pearson
    scenario.noise = noise
    return data, scenario

In [ ]:
for scenario in scenario_list[:1]:
    nnts.torch.data.datasets.seed_everything(scenario.seed)
    df, scenario = prepare(df_orig.copy(), scenario)
    split_data = splitter.split(
        df, trn_length, val_test_length, val_test_length
    )
    trn_dl, val_dl, test_dl = nnts.data.map_to_dataloaders(
        split_data,
        metadata,
        scenario,
        params,
        nnts.torch.data.TorchTimeseriesDataLoaderFactory(),
        [nnts.torch.data.preprocessing.StandardScaler()],
    )
    logger = nnts.loggers.WandbRun(
        project=f"{model_name}-{metadata.dataset}",
        name=scenario.name,
        config={
            **params.__dict__,
            **metadata.__dict__,
            **scenario.__dict__,
        },
        path=PATH,
    )

    net = nnts.torch.models.SegLSTM(
        nnts.torch.models.LinearModel,
        params,
        nnts.torch.data.preprocessing.masked_mean_abs_scaling,
        scenario.covariates + 1,
        24
    )
    trner = trainers.TorchEpochTrainer(
        nnts.models.TrainerState(), 
        net, 
        params, 
        metadata, 
        os.path.join(PATH, f"{scenario.name}.pt"),
    )
    logger.configure(trner.events)

    evaluator = trner.train(trn_dl, val_dl)
    handle = net.decoder.register_forward_hook(logger.log_activations)
    y_hat, y = evaluator.evaluate(
        test_dl, scenario.prediction_length, metadata.context_length, hooks=handle
    )
    handle.remove()
    test_metrics = nnts.metrics.calc_metrics(
        y_hat, y, trn_dl, metadata
    )
    logger.log(test_metrics)
    logger.finish()

In [ ]:
y_hat.shape, y.shape

In [ ]:
nnts.metrics.calc_metrics(y_hat, y, trn_dl, metadata)

In [ ]:
nnts.metrics.calc_metrics(y_hat[:, :1, :], y[:, :1, :], trn_dl, metadata)

In [ ]:
def save_results(y_hat, y, name):
    torch.save(y_hat, f"{PATH}/{name}_y_hat.pt")
    torch.save(y, f"{PATH}/{name}_y.pt")
save_results(y_hat, y, scenario.name)

In [ ]:
covariate_name = f"cov-1-pearsn-0.68-pl-{str(scenario.prediction_length)}-seed-{scenario.seed}"
covariate_y_hat = torch.load(f"{PATH}/{covariate_name}_y_hat.pt")
covariate_y = torch.load(f"{PATH}/{covariate_name}_y.pt")

In [ ]:
def calculate_forecast_horizon_metrics(y_hat, y, metadata, metric="mae"):
    forecast_horizon_metrics = []
    for i in range(1, metadata.prediction_length):
        metrics = nnts.metrics.calc_metrics(y[:, :i, :], y_hat[:, :i, :], metadata.freq, metadata.seasonality)
        forecast_horizon_metrics.append(metrics[metric])
    return forecast_horizon_metrics

forecast_horizon_metrics = calculate_forecast_horizon_metrics(y_hat, y, metadata, "mae")
covariate_forecast_horizon_metrics = calculate_forecast_horizon_metrics(covariate_y_hat, covariate_y, metadata, "mae")

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
plt.plot(forecast_horizon_metrics, label='univariate')
plt.plot(covariate_forecast_horizon_metrics, label='covariate (0.68)')
plt.xlabel("Forecast Horizon")
plt.ylabel("Error (MAE)")
plt.legend()
plt.show()

In [ ]:
csv_aggregator = nnts.pandas.CSVFileAggregator(PATH, "results")
results = csv_aggregator()

In [ ]:
df.tail(metadata.prediction_length*50)['y'].plot()

In [ ]:
y_hat_last = y_hat[:, :1, ...]
y_last = y[:, :1, ...]
df_test = df.tail(y_hat_last.shape[0])
df_test["y_check"] = y_last.squeeze()
df_test["y_hat"] = y_hat_last.squeeze()
df_test[["y", "y_check", "y_hat"]]
df_test.set_index("ds")[["y_check", "y_hat"]].iloc[4500:4500+336].plot(figsize=(20, 10))

In [ ]:
test_metrics = nnts.metrics.calc_metrics(y_last, y_hat_last, metadata.freq, metadata.seasonality)
test_metrics